#### processes cipher text received from the curl command and converts it to its hex representation using the mapping

In [1]:
import numpy as np
import random
from numpy.linalg import matrix_rank
# import eaeae_helper

# np.set_printoptions(threshold=np.nan)
ciphertexts_file = open("ciphertextsP0P4.txt","r")

hex_ctexts = [[] for i in range(128)]
ctexts = [[] for i in range(128)]

for p1 in range(128):
    for p2 in range(128):
        unprocessed_ctext = ciphertexts_file.read(34)
        ctext = unprocessed_ctext.replace('{\"ciphertext\": \"',"")
        ctext = ctext.replace('\"}','')
        ctexts[p1].append(ctext)
        hex_ctext = ""
        for char in ctext:
            nibble = ord(char) - ord('f')
            hex_ctext += hex(nibble)[2:]
        hex_ctexts[p1].append(hex_ctext)
        
ciphertexts_file.close()

#### solves the system of homogenous equations E3

In [2]:
# generate 8 sets of num_sets homogenous equations
num_sets = 256
equation_set = np.zeros((8, num_sets, 128), dtype = int)

# for the first set of 128 equations
for ebox in range(8):
    for p1 in range(128):
        for p2 in range(128):
            var_index = int(hex_ctexts[p1][p2][2*ebox:2*ebox+2], 16)
            assert var_index < 128
            equation_set[ebox][p1][var_index] ^= 1
# for the second set of 128 equations
for ebox in range(8):
    for p2 in range(num_sets-128):
        for p1 in range(128):
            var_index = int(hex_ctexts[p1][p2][2*ebox:2*ebox+2], 16)
            assert var_index < 128
            equation_set[ebox][128+p2][var_index] ^= 1

# solve the system of equations
free_variables = [[] for ebox in range(8)]
for ebox in range(8):
    for var in range(128):    
        # identify the row with the next pivot
        if np.any(equation_set[ebox][var:,var] == 1):
            pivot_row = np.where(equation_set[ebox][var:,var] == 1)[0][0] + var
        else:
            # remember the free variables
            free_variables[ebox].append(var)
            continue
        # swap the pivot row into place
        equation_set[ebox][[var, pivot_row]] = equation_set[ebox][[pivot_row, var]]
        # make the elements above and below the pivot zero
        for eq in range(num_sets):
            if equation_set[ebox][eq][var] == 1:
                if eq != var:
                    equation_set[ebox][eq] ^= equation_set[ebox][var]
    
for ebox in range(8):
    print("for ebox {}, rank: {}".format(ebox, 128-len(free_variables[ebox])))

for ebox 0, rank: 1
for ebox 1, rank: 1
for ebox 2, rank: 1
for ebox 3, rank: 1
for ebox 4, rank: 120
for ebox 5, rank: 120
for ebox 6, rank: 120
for ebox 7, rank: 120
